In [1]:
# !pip install webvtt-py spacy sentence-transformers qdrant-client bertopic gradio ffmpeg-python
# !python -m spacy download en_core_web_sm

In [2]:
import os, glob
from webvtt import WebVTT

In [3]:
def time_to_seconds(timestamp):
    h, m, s = timestamp.split(':')
    return int(h) * 3600 + int(m) * 60 + float(s)

captions_data = []  # List of dicts: {'video_id', 'start', 'end', 'text'}
for video_dir in sorted(os.listdir("./data/videos")):
    video_path = os.path.join("./data/videos", video_dir)
    if not os.path.isdir(video_path):
        continue
    # Find English VTT file
    vtt_files = glob.glob(os.path.join(video_path, "*.en.vtt"))
    if not vtt_files:
        continue
    vtt_path = vtt_files[0]
    # Parse captions
    for caption in WebVTT().read(vtt_path):
        text = caption.text.strip().replace("\n", " ")
        if not text:
            continue
        start = time_to_seconds(caption.start)
        end = time_to_seconds(caption.end)
        captions_data.append({"video_id": video_dir, "start": start, "end": end, "text": text})

# Sort by video and time
captions_data.sort(key=lambda x: (x['video_id'], x['start']))

# Deduplicate consecutive segments
filtered_caps = []
prev_text = None
prev_vid = None
for cap in captions_data:
    if cap["video_id"] != prev_vid:
        prev_text = None  # reset at new video
        prev_vid = cap["video_id"]
    if cap["text"] != prev_text:
        filtered_caps.append(cap)
    prev_text = cap["text"]
captions_data = filtered_caps

len(captions_data), "caption segments loaded."


(6970, 'caption segments loaded.')

In [4]:
# # from google.colab import drive
# # drive.mount('/content/drive')
# !cp -r /content/drive/MyDrive/data /content

In [5]:
import spacy
nlp = spacy.blank("en")
nlp.add_pipe("sentencizer")

from itertools import groupby

sentences = []  # List of dicts: {'video_id', 'start', 'end', 'sentence'}
for video_id, group in groupby(captions_data, key=lambda x: x['video_id']):
    group = list(group)
    combined_text = " ".join([seg["text"] for seg in group])
    doc = nlp(combined_text)
    # Compute cumulative character lengths for segment boundaries
    cum_lengths = [0]
    for seg in group:
        cum_lengths.append(cum_lengths[-1] + len(seg["text"]) + 1)
    for sent in doc.sents:
        sent_text = sent.text.strip()
        if not sent_text:
            continue
        start_char = sent.start_char
        end_char = sent.end_char
        # Find segment indices containing sentence start/end
        start_idx = max(i for i, length in enumerate(cum_lengths) if length <= start_char)
        end_idx = max(i for i, length in enumerate(cum_lengths) if length < end_char)
        start_time = group[start_idx]["start"]
        end_time = group[end_idx]["end"]
        sentences.append({
            "video_id": video_id,
            "start": start_time,
            "end": end_time,
            "sentence": sent_text
        })

print(f"Created {len(sentences)} merged sentences from captions.")


Created 8 merged sentences from captions.


In [10]:
# !pip install accelerate>=0.21.0
# !pip install -U transformers
# !pip install peft==0.3.0
# !pip install datasets
# !pip install "sentence-transformers[train]"
!pip install accelerate --upgrade
!pip install peft --upgrade
!pip install transformers --upgrade
from itertools import groupby
from torch.utils.data import DataLoader
import spacy
import random
# Import the datasets library and the Dataset class
# !pip install --upgrade sentence-transformers datasets

nlp = spacy.blank("en")
nlp.add_pipe("sentencizer")

def windowize_sentence(sentence, window_size=50, stride=50):
    tokens = sentence.split()
    spans = []
    # slide a window of up to 50 words (no overlap, since stride=window_size)
    for start in range(0, max(1, len(tokens) - window_size + 1), stride):
        spans.append(" ".join(tokens[start:start + window_size]))
    if not spans:
        spans = [" ".join(tokens)]
    return spans

processed = []
for vid, group in groupby(sentences, key=lambda x: x['video_id']):
    group = list(group)
    long_para = group[0]['sentence']
    for sent in nlp(long_para).sents:
        # further split any spaCy sentence over 50 words
        for chunk in windowize_sentence(sent.text, window_size=50, stride=50):
            processed.append({
                "video_id": vid,
                "sentence": chunk.strip()
            })

train_examples = []
for vid, group in groupby(processed, key=lambda x: x['video_id']):
    grp = list(group)
    for i in range(len(grp) - 1):
        s1, s2 = grp[i]['sentence'], grp[i+1]['sentence']
        if not s1 or not s2:
            continue
        # import here, where InputExample is used
        from sentence_transformers import InputExample
        train_examples.append(InputExample(texts=[s1, s2]))

# 4. Shuffle, batch, and fine‐tune
random.shuffle(train_examples)
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

# import here, where SentenceTransformer is used
from sentence_transformers import SentenceTransformer, losses
model = SentenceTransformer('all-MiniLM-L6-v2')
train_loss = losses.MultipleNegativesRankingLoss(model=model)
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=1,
    warmup_steps=100
)

# 5. Save your fine‐tuned model
model.save("fine_tuned_minilm")

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: yc6317 (project_hpml_yuzhong) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss


In [16]:
# !pip install peft --upgrade
from peft import PeftModelForFeatureExtraction
sent_texts = [
    chunk
    for s in sentences
    for sent in nlp(s["sentence"]).sents  # Split into spaCy sentences
    for chunk in windowize_sentence(sent.text)  # Further split into windowed chunks
]
embeddings = model.encode(sent_texts, convert_to_numpy=True)
print("Encoded sentences to vectors of shape:", embeddings.shape)



Encoded sentences to vectors of shape: (1435, 384)


In [17]:
from bertopic import BERTopic

print(len(sent_texts))
topics, probs = topic_model.fit_transform(sent_texts)
print(topic_model.get_topic_info().head())

1435
   Topic  Count                              Name  \
0     -1    495                   -1_uh_the_of_to   
1      0    178        0_feature_map_depth_filter   
2      1     87  1_layers_layer_convolutional_are   
3      2     58             2_image_images_uh_car   
4      3     50          3_signal_was_strength_of   

                                      Representation  \
0     [uh, the, of, to, is, and, this, that, in, we]   
1  [feature, map, depth, filter, the, output, inp...   
2  [layers, layer, convolutional, are, the, uh, s...   
3  [image, images, uh, car, in, um, of, that, you...   
4  [signal, was, strength, of, uh, the, this, wer...   

                                 Representative_Docs  
0  [uh to be called the radar problem and uh we'l...  
1  [input feature map of depth map of depth map o...  
2  [in the fully connected dense layer fully conn...  
3  [in they are uh positioned at some point in th...  
4  [the radar problem High signal strength low si...  


In [25]:
!pip install qdrant-client
import os
import time
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams, PointStruct


# Set Qdrant storage path to be within the current working directory
os.environ["QDRANT_STORAGE_PATH"] = "./qdrant_storage"

# Start the Qdrant server in the background
client = QdrantClient(host="localhost", port=6333)
print("Qdrant server started")

Qdrant server started


In [38]:
from qdrant_client import QdrantClient, models
from qdrant_client.http.models import PointStruct

client = QdrantClient(host="localhost", port=6333)
collection_name = "video_captions"
vector_dim = embeddings.shape[1]
vectors_config = models.VectorParams(size=vector_dim, distance=models.Distance.COSINE)
client.recreate_collection(collection_name=collection_name, vectors_config=vectors_config)

# Prepare points for upserting
points = []
for i, sent in enumerate(sentences):
    payload = {
        "video_id": sent["video_id"],
        "start": sent["start"],
        "end": sent["end"],
        "sentence": sent["sentence"]
    }
    points.append(PointStruct(id=i, vector=embeddings[i].tolist(), payload=payload))

client.upsert(collection_name=collection_name, points=points)
print(f"Uploaded {len(points)} points to Qdrant collection '{collection_name}'.")


Connected to localhost successfully!
Connection failed: [Errno 99] Cannot assign requested address


In [32]:
def answer_query(query, top_k=1):
    q_vec = model.encode([query], convert_to_numpy=True)[0]
    results = client.search(collection_name=collection_name, query_vector=q_vec, limit=top_k)
    top = results[0]  # best match
    data = top.payload
    return data["video_id"], data["start"], data["end"], data["sentence"]

# Example query
video_id, start, end, sentence = answer_query("Explain how convolutional layers work")
print(f"Top answer: \"{sentence}\" (Video {video_id}, {start:.1f}-{end:.1f}s)")


ResponseHandlingException: [Errno 99] Cannot assign requested address

In [ ]:
import ffmpeg

def extract_clip(video_id, start, end, output_path="clip.mp4"):
    # Find the video file for this ID
    mp4_files = glob.glob(os.path.join("./data/videos", video_id, "*.mp4"))
    if not mp4_files:
        raise FileNotFoundError(f"No video found for ID {video_id}")
    video_path = mp4_files[0]
    (
        ffmpeg
        .input(video_path, ss=start, to=end)
        .output(output_path, codec="copy")
        .run(overwrite_output=True)
    )
    return output_path

# Example: extract the clip for the top result
clip_file = extract_clip(video_id, start, end, output_path=f"clip_{video_id}_{int(start)}_{int(end)}.mp4")
print(f"Saved clip to {clip_file}")


In [ ]:
%load_ext gradio

%%blocks
import gradio as gr

def qa_pipeline(question):
    vid, s, e, ans = answer_query(question)
    clip = extract_clip(vid, s, e, output_path=f"clip_{vid}_{int(s)}_{int(e)}.mp4")
    return clip, ans

with gr.Blocks() as demo:
    gr.Markdown("## Chat with Your Video Library")
    inp = gr.Textbox(label="Enter your question")
    vid_out = gr.Video(label="Relevant clip")
    text_out = gr.Textbox(label="Answer sentence")
    inp.submit(qa_pipeline, [inp], [vid_out, text_out])

demo
